In [6]:
import sys
sys.path.append("..")

from pytorch.data import normalize, crop_upper_part
from pytorch.model import LModel
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn import metrics
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

In [2]:
ANNOTATED_TEST_DATASET = '../data/test_annotated'
MODEL_PATH = "../models/epoch_14-valLoss_0.00067-valAcc_1.00000"
CPU_CORES = 6
BATCH_SIZE = 32
NUM_CLASSES = 26
USE_GPU = True

In [7]:
# THIS REQUIRES THE master branch of the `torchivsion` package
def data_transformations(input_shape, crop_perc = 0.5):
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.Resize((input_shape[1], input_shape[2])),
        transforms.Lambda(lambda x: normalize(np.array(x, dtype=np.float32))),
        transforms.ToTensor(),
    ])

def var(tensor, volatile=False):
    if USE_GPU:
        tensor = tensor.cuda(0)
    return Variable(tensor, volatile=volatile)

def test(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.softmax(logit, dim = 0).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), ground_truths
    

In [36]:
# Load model
model = LModel(num_classes=NUM_CLASSES, fine_tune=True, margin=1)


model.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)


if USE_GPU:
    model.cuda(0)

Loaded model from: ../models/epoch_14-valLoss_0.00067-valAcc_1.00000


/home/bartol/.virtualenvs/mozgalo/lib/python3.5/site-packages/torch/nn/modules/module.py:514: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


In [27]:
test_transform = data_transformations(model.model.input_size)
test_dataset = datasets.ImageFolder(root=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=CPU_CORES)

In [56]:
pred, true = test(model, test_dataset_loader)

In [86]:
print(true[:100])

np.max(pred, 1)[:100]

[19, 4, 25, 22, 25, 25, 10, 25, 25, 3, 5, 24, 25, 9, 20, 2, 10, 23, 19, 25, 1, 25, 2, 25, 25, 25, 3, 25, 23, 0, 25, 25, 20, 25, 21, 25, 19, 25, 25, 25, 25, 13, 24, 3, 2, 25, 25, 25, 25, 20, 25, 10, 25, 14, 25, 15, 25, 25, 25, 25, 23, 18, 12, 25, 19, 25, 25, 20, 25, 1, 25, 7, 9, 25, 3, 25, 1, 25, 24, 9, 7, 17, 25, 11, 25, 25, 25, 12, 25, 25, 23, 12, 25, 25, 0, 25, 9, 25, 25, 25]


array([0.57844186, 0.9477168 , 0.5854987 , 0.9999529 , 0.08573863,
       0.08208578, 0.16463608, 0.04125353, 0.3600355 , 0.04967184,
       0.9998759 , 0.999345  , 0.74939835, 0.9998833 , 0.9977604 ,
       0.41321862, 0.07723735, 0.99175185, 0.4214487 , 0.05868314,
       0.99982446, 0.02958403, 0.58677584, 0.847426  , 0.08206109,
       0.2049164 , 0.99843603, 0.16592279, 0.0430133 , 0.9969772 ,
       0.21745123, 0.14201131, 0.20205298, 0.07701006, 0.9956423 ,
       0.49463594, 0.9999417 , 0.16497746, 0.41928592, 0.0656886 ,
       0.07937535, 0.99851674, 0.9996972 , 0.99993956, 0.99995756,
       0.13273172, 0.21274583, 0.11977648, 0.07522453, 0.9998869 ,
       0.2561855 , 0.65320385, 0.3185851 , 0.97605693, 0.33505952,
       0.99741435, 0.07496938, 0.09490413, 0.43003523, 0.03731275,
       0.9994991 , 0.9348558 , 0.999551  , 0.6912993 , 0.9995492 ,
       0.08941256, 0.21459186, 0.99997234, 0.2554902 , 0.93372065,
       0.60944295, 0.10903376, 0.2164551 , 0.04589016, 0.99995

In [115]:
np.mean(np.max(pred, 1))

0.49984625

In [117]:
thr = 0.9
predicted = []
for act in pred:
    sort = sorted(act)[::-1]
    
    if sort[0] < thr:
        predicted.append(25)
    else:
        predicted.append(np.argmax(act))
        
report = metrics.classification_report(true, predicted)
print(report)

             precision    recall  f1-score   support

          0       0.91      0.58      0.71       200
          1       1.00      0.60      0.75       200
          2       1.00      0.60      0.75       200
          3       1.00      0.67      0.80       200
          4       0.98      0.65      0.78       198
          5       0.99      0.59      0.74       202
          6       0.97      0.63      0.76       200
          7       0.97      0.56      0.71       200
          8       0.96      0.61      0.75       198
          9       0.82      0.58      0.68       201
         10       1.00      0.36      0.53       198
         11       0.97      0.57      0.72       200
         12       0.98      0.63      0.77       200
         13       0.99      0.68      0.80       198
         14       0.98      0.60      0.75       198
         15       1.00      0.64      0.78       200
         16       0.97      0.61      0.75       207
         17       0.98      0.55      0.71   